In [1]:
from dotenv import dotenv_values
config = dotenv_values("../.env")

In [26]:
OPENROUTER_API_KEY = config.get('OPENROUTER_API_KEY')
OPENROUTER_API_HOST = config.get('OPENROUTER_API_HOST')
STT_MODEL="openai/gpt-audio-mini"

In [27]:
OPENROUTER_API_HOST

'https://openrouter.ai/api/v1'

In [30]:
from openai import OpenAI
import base64
import wave

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY
)

response = client.chat.completions.create(
    model=STT_MODEL,
    modalities=["text", "audio"],
    stream=True,
    messages=[
        {
            "role": "system",
            "content": "Repat with your voice what user sayings"
        },
        {
            "role": "user",
            "content": "Hello, how are you?"
        }
    ],
    audio={
        "voice": "alloy",
        "format": "pcm16"
    }
)

text_response = ""
audio_chunks = []
transcript = ""
audio_id = None

print("Streaming response...\n")

for chunk in response:
    # Collect text content
    if chunk.choices[0].delta.content:
        text_content = chunk.choices[0].delta.content
        text_response += text_content
        print(text_content, end="", flush=True)
    
    # Collect audio data and transcript
    if hasattr(chunk.choices[0].delta, 'audio') and chunk.choices[0].delta.audio:
        audio_dict = chunk.choices[0].delta.audio
        
        # audio_dict is a dictionary
        if isinstance(audio_dict, dict):
            # Get audio ID
            if 'id' in audio_dict and audio_dict['id']:
                audio_id = audio_dict['id']
            
            # Get audio data
            if 'data' in audio_dict and audio_dict['data']:
                try:
                    decoded_audio = base64.b64decode(audio_dict['data'])
                    audio_chunks.append(decoded_audio)
                except Exception as e:
                    print(f"\nError decoding audio: {e}")
            
            # Get transcript
            if 'transcript' in audio_dict and audio_dict['transcript']:
                transcript += audio_dict['transcript']
                print(audio_dict['transcript'], end="", flush=True)

print("\n\nProcessing audio...")

# Combine all audio chunks
if audio_chunks:
    audio_data = b"".join(audio_chunks)
    
    # Save as WAV file
    try:
        with wave.open("output.wav", "wb") as wav_file:
            wav_file.setnchannels(1)  # Mono
            wav_file.setsampwidth(2)  # 16-bit
            wav_file.setframerate(24000)  # 24kHz
            wav_file.writeframes(audio_data)
        
        print(f"Audio saved to output.wav ({len(audio_data)} bytes)")
    except Exception as e:
        print(f"Error saving WAV file: {e}")
        
        # Save as raw PCM if WAV fails
        with open("output.pcm", "wb") as f:
            f.write(audio_data)
        print(f"Saved as raw PCM: output.pcm ({len(audio_data)} bytes)")
else:
    print("No audio data received")

if audio_id:
    print(f"\nAudio ID: {audio_id}")

if transcript:
    print(f"\nFull transcript: {transcript}")

if text_response:
    print(f"\nText response: {text_response}")

Processing...
Hey there! I'm doing well, thanks for asking. How about you?

Audio saved to output.wav (204000 bytes)
Processing...
Thanks for the test message! Everything seems to be working fine. Let me know if there's anything specific you'd like to talk about!

Audio saved to test.wav (328800 bytes)


'test.wav'

In [20]:
from openai import OpenAI
import base64
import wave

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY
)

def text_to_speech(text, output_file="output.wav", voice="alloy"):
    """Convert text to speech and save as WAV file"""
    
    response = client.chat.completions.create(
        model="openai/gpt-4o-audio-preview",
        modalities=["text", "audio"],
        stream=True,
        messages=[
            {
                "role": "user",
                "content": text
            }
        ],
        audio={
            "voice": voice,
            "format": "pcm16"
        }
    )
    
    audio_chunks = []
    transcript = ""
    
    print("Processing...")
    
    for chunk in response:
        if hasattr(chunk.choices[0].delta, 'audio') and chunk.choices[0].delta.audio:
            audio_dict = chunk.choices[0].delta.audio
            
            if isinstance(audio_dict, dict):
                if 'data' in audio_dict and audio_dict['data']:
                    decoded_audio = base64.b64decode(audio_dict['data'])
                    audio_chunks.append(decoded_audio)
                
                if 'transcript' in audio_dict and audio_dict['transcript']:
                    transcript += audio_dict['transcript']
                    print(audio_dict['transcript'], end="", flush=True)
    
    if audio_chunks:
        audio_data = b"".join(audio_chunks)
        
        with wave.open(output_file, "wb") as wav_file:
            wav_file.setnchannels(1)
            wav_file.setsampwidth(2)
            wav_file.setframerate(24000)
            wav_file.writeframes(audio_data)
        
        print(f"\n\nAudio saved to {output_file} ({len(audio_data)} bytes)")
        return output_file
    else:
        print("\nNo audio data received")
        return None

# Usage
text_to_speech("Hello, how are you?")

# Different voice
text_to_speech("This is a test message", output_file="test.wav", voice="nova")

Audio saved to output.wav

Text response: 


In [21]:
audio_data

b''